# Лабораторная работа №6
*Вариант 2*

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math, Markdown
import mylib

## Задача 1
Вычислить значение интеграла $I=\int\limits_0^2 P_m (x)dx$, где $P_m(x) = \sum\limits_{i=0}^m c_i x^i$, с помощью квадратурных формул левых прямоугольников и по формуле трапеций с точностью $\varepsilon = 10^{-14}$ . Вычислить интеграл по формуле Гаусса. 

In [2]:
# Условие
coeffs = np.array([1, 0.9, 0.8, 0.7, 0, 2.2, -4.1, 0, -3.4, 3.5])

def get_polynom(coeffs=coeffs):
    coeffs = np.asarray(coeffs)
    m = len(coeffs)
    def polynom_func(x):
        x = np.asarray(x)
        if x.ndim == 0:
            x_powers = x ** np.arange(0, m)
        else:
            x_powers = x[:, np.newaxis] ** np.arange(0, m)
        return np.dot(x_powers, coeffs)
    return polynom_func

P = get_polynom() 
eps = 10**-14
a = 0
b = 2


display(Markdown(fr"""
Условие:   
$\quad\quad P_{{{len(coeffs)-1}}} = {" ".join(f"{coeffs[i]:+} x^{i}"for i in range(len(coeffs)))}$  
$\quad\quad \varepsilon = {eps}$  
$\quad\quad a = {a}$  
$\quad\quad b = {b}$  
"""))


Условие:   
$\quad\quad P_{9} = +1.0 x^0 +0.9 x^1 +0.8 x^2 +0.7 x^3 +0.0 x^4 +2.2 x^5 -4.1 x^6 +0.0 x^7 -3.4 x^8 +3.5 x^9$  
$\quad\quad \varepsilon = 1e-14$  
$\quad\quad a = 0$  
$\quad\quad b = 2$  


In [3]:
# Аналитическое решение
coeffs_anti = coeffs * (1/np.arange(1, len(coeffs) + 1))
antiderivative = get_polynom(coeffs_anti) # Первообразная
I = antiderivative(b) - antiderivative(a) # Формула Ньютона Лейбница

mylib.show_large_number(f"{I:.17}", description="Аналитическое значение интеграла:")

In [10]:
# Вычисление количества шагов для достижения заданной точности
coeffs_prime = coeffs[1:] * np.arange(1, len(coeffs))
P_prime = get_polynom(coeffs_prime)
coeffs_double = coeffs_prime[1:] * np.arange(1, len(coeffs_prime))
P_double = get_polynom(coeffs_double)

points_to_check = np.linspace(a, b, int(1e6))
M1 = np.max(P_prime(points_to_check))
M2 = np.max(P_double(points_to_check))

n_left_rect = int(np.ceil((a + b)**2 * M1 / (2 * eps)))
n_trap = int(np.ceil(np.sqrt((a + b) ** 3 * M2 / (12 * eps))))

display(mylib.show_large_number(f"{n_trap}", "Число шагов для метода трапеций:"))

display(mylib.show_large_number(f"{n_left_rect}", "Число шагов для метода левых прямоугольников:"))

In [ ]:
# 